In [5]:
import pandas
import numpy as np

In [6]:
file_name = r'data/hackerrank_email/training_dataset.csv'
data = pandas.read_csv(file_name)
print(data.shape)

(486048, 54)


In [7]:
data.head(10)

,user_id,mail_id,mail_category,mail_type,sent_time,open_time,click_time,unsubscribe_time,last_online,hacker_created_at,...,submissions_count_contest_1_days,submissions_count_contest_30_days,submissions_count_contest_365_days,submissions_count_contest_7_days,submissions_count_master,submissions_count_master_1_days,submissions_count_master_30_days,submissions_count_master_365_days,submissions_count_master_7_days,unsubscribed
0,AR+tMy3H/E+Re8Id20zUIz+amJkv6KU12o+BrgIDin0=,DQ/4I+GIOz2ZoIiK0Lg0AkwnI35XotghgUK/MYc101I=,mail_category_2,mail_type_1,1463497837,1.463541e+09,NaN,NaN,1.459520e+09,1432533023,...,0,0,0,0,13,0,0,13,0,False
1,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1461357640,NaN,NaN,NaN,1.461210e+09,1432184291,...,0,3,16,0,83,0,43,83,4,False
2,OEfFUcsTAGInCfsHuLZuIgdSNtuNsg8EdfN98VUZVTs=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1463499639,NaN,NaN,NaN,1.463411e+09,1433045937,...,0,0,0,0,16,0,3,16,0,False
3,1P4AOvdzJzhDSHi7jJ3udWv4ajpKxOn4T/rCLv4PrXU=,EHNBRbi6i9KO6cMHsuDPFjZVp2cY3RH+BiOKwPwzLQs=,mail_category_1,mail_type_1,1463182983,NaN,NaN,NaN,1.462768e+09,1432184291,...,0,0,16,0,85,0,9,85,0,False
4,CYRcuV0cR0algMZJ1N6+3uKcqi8iu+6tJNzmBbmgN7o=,K0y/NW59TJkYc5y0HUwDeAXrewYT0JQlkcozz0s2V5Q=,mail_category_4,mail_type_1,1461855019,NaN,NaN,NaN,1.461248e+09,1432998058,...,0,1,17,0,43,0,0,43,0,False
5,UREtRIcAzBFKmMt9ZSkFLofOWKAKmzsDD7BwrdxcPuk=,UGATDXARg7jMEInKH7oXgty2nwxnwD2l0OW/8Nsa0MI=,mail_category_6,mail_type_1,1459927021,NaN,NaN,NaN,1.457962e+09,1431945704,...,0,0,13,0,88,0,0,88,0,False
6,NiDHS4AU/QOs3ppF618O6bUg8BxJd+EV4Ek26KWrDc0=,qE9zgWiITYA97RUiN4X/t9IVWLViLz+lKijaYegyBiQ=,mail_category_1,mail_type_1,1460230930,1.460232e+09,NaN,NaN,1.460135e+09,1432658834,...,0,0,0,0,20,0,12,20,12,False
7,sGNmP4gGVwh3YBpXtq6ZI05PIa3Z0iUaF+ujKonctu0=,BL3z4RtiyfIDydaRYWX2ZXL6IX10QH1yG5ak1s/8Lls=,mail_category_1,mail_type_1,1462282973,NaN,NaN,NaN,1.462193e+09,1432294499,...,0,0,0,0,14,0,14,14,14,False
8,nyoFu8qU0B5viJDXfrVRQ65ytOSSSY3c7okcjo1Z76M=,4+EEK8RbNYwuFCHznY9XSRCV4Yek60bHVgnP3jtjjzk=,mail_category_13,mail_type_2,1462888545,1.462890e+09,NaN,NaN,1.462860e+09,1432795707,...,0,19,265,0,129,0,0,129,0,False
9,A2EUbD7QN5SXpJ7FhPY5Ne7zwjwYhRvAheGskLl0Os8=,4+EEK8RbNYwuFCHznY9XSRCV4Yek60bHVgnP3jtjjzk=,mail_category_13,mail_type_2,1462888549,NaN,NaN,NaN,1.462369e+09,1432059719,...,0,0,0,0,3,0,0,3,0,False


In [17]:
print(data.shape[0])
opened = (len(data[data['opened'] == True])) 
not_opened = (len(data[data['opened'] == False]))
print ('Opened: ' + str(opened) + '; Unopened: ' + str(not_opened) + '')

486048
Opened: 161347; Unopened: 324701


In [18]:
data_sub = data[['clicked', 'unsubscribed', 'submissions_count_master_7_days', 'contest_login_count_30_days', 'contest_login_count_7_days',
'hacker_confirmation','opened']]
data_sub.corr(method='pearson')

,clicked,unsubscribed,submissions_count_master_7_days,contest_login_count_30_days,contest_login_count_7_days,hacker_confirmation,opened
clicked,1.000000,-0.006387,0.063515,0.061719,0.061998,0.025972,0.308118
unsubscribed,-0.006387,1.000000,-0.003167,-0.002335,0.007434,0.001697,0.075115
submissions_count_master_7_days,0.063515,-0.003167,1.000000,0.176027,0.222613,0.046932,0.054511
contest_login_count_30_days,0.061719,-0.002335,0.176027,1.000000,0.546756,0.003917,0.048865
contest_login_count_7_days,0.061998,0.007434,0.222613,0.546756,1.000000,-0.025393,0.043133
hacker_confirmation,0.025972,0.001697,0.046932,0.003917,-0.025393,1.000000,0.076434
opened,0.308118,0.075115,0.054511,0.048865,0.043133,0.076434,1.000000


In [19]:
#list(data.columns.values)

In [20]:
#data.dtypes